In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from keras.utils.np_utils import to_categorical
from keras.models import Sequential,load_model
from keras.layers import Dense, Dropout, Flatten, Lambda, Activation
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam

train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

In [4]:
def std_img(x):
    for i in range(3):
        x[:, :, i] -= np.mean(x[:, :, i].flatten())
        x[:, :, i] /= np.std(x[:, :, i].flatten()) + 1e-7
    return x

def get_image(df):
    '''Create 3-channel 'images'. Return rescale-normalised images.'''
    images = []
    for i, row in df.iterrows():
        # Formulate the bands as 75x75 arrays
        band_1 = np.array(row['band_1']).reshape(75, 75)
        band_2 = np.array(row['band_2']).reshape(75, 75)
        band_3 = (band_1 + band_2)/2

        # Rescale
        img = np.dstack([band_1,band_2,band_3])
        img = std_img(img)

        images.append(img)
    return np.array(images)


train_x = get_image(train_df)
test_x = get_image(test_df)

print(train_x.shape,test_x.shape)

(1604, 75, 75, 3) (8424, 75, 75, 3)


In [5]:
y = train_df.is_iceberg.values
print(y[:5])

[0 0 1 0 0]


In [6]:
from keras.callbacks import ModelCheckpoint
def create_model():
    '''Create the FCN and return a keras model.'''

    model = Sequential()
    model.add(Conv2D(32, (3, 3),input_shape=(75, 75, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(32, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(ZeroPadding2D((1, 1)))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu', strides=1))
    model.add(MaxPooling2D((2, 2), strides=(2, 2)))
    
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))

    model.add(Dense(1, activation='sigmoid'))
    return model
print('model model')

def rot_aut(Xtr,ytr):
    # aug on train
    data_cnt = len(ytr)
    print(data_cnt)
    aug_X = []
    aug_y = []

    for i in range(data_cnt):
        img = Xtr[i]
        tmp_y = ytr[i]

        # org img
        aug_X.append(img)
        aug_y.append(tmp_y)

        # flip
        tmp_img = np.fliplr(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.flipud(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)

        tmp_img = np.rot90(img)
        aug_X.append(tmp_img)
        aug_y.append(tmp_y)
    return np.array(aug_X),np.array(aug_y)

model model


In [7]:
from sklearn.model_selection import KFold
def kfold_train(fold_cnt=3,rnd=42):
    train_pred, test_pred = np.zeros((1604,1)),np.zeros((8424,1))
    kf = KFold(n_splits=fold_cnt, shuffle=True, random_state=2*rnd)
    for train_index, test_index in kf.split(train_x):
        curr_x,curr_y = train_x[train_index],y[train_index]
        curr_x,curr_y = rot_aut(curr_x,curr_y)
        val_x,val_y = train_x[test_index],y[test_index]
        
        model = create_model()
        model.compile(loss='binary_crossentropy', optimizer=Adam(0.0005), metrics=['accuracy'])
        model_p = 'best_m.h5'
        model_chk = ModelCheckpoint(filepath=model_p, monitor='val_loss', save_best_only=True, verbose=1)
        model.fit(curr_x,curr_y,
                  validation_data=(val_x,val_y),
                  batch_size=64, epochs=30, 
                  verbose=2,
                  callbacks=[model_chk]
                 )
        model = load_model(model_p)
        
        train_pred[test_index] = model.predict(val_x)
        test_pred = test_pred + model.predict(test_x)/fold_cnt
    return train_pred,test_pred

train_pred,test_pred = kfold_train(fold_cnt=4)

1203
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 73, 73, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 38, 38, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 36, 36, 32)        9248      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 34, 34, 32)        9248      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 32)        0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 19, 19, 32)        0         
_____

Train on 4812 samples, validate on 401 samples
Epoch 1/30
Epoch 00001: val_loss improved from inf to 0.40008, saving model to best_m.h5
 - 4s - loss: 0.5097 - acc: 0.7267 - val_loss: 0.4001 - val_acc: 0.8105
Epoch 2/30
Epoch 00002: val_loss improved from 0.40008 to 0.31942, saving model to best_m.h5
 - 3s - loss: 0.3436 - acc: 0.8437 - val_loss: 0.3194 - val_acc: 0.8504
Epoch 3/30
Epoch 00003: val_loss improved from 0.31942 to 0.30277, saving model to best_m.h5
 - 3s - loss: 0.2925 - acc: 0.8707 - val_loss: 0.3028 - val_acc: 0.8628
Epoch 4/30
Epoch 00004: val_loss improved from 0.30277 to 0.27635, saving model to best_m.h5
 - 3s - loss: 0.2563 - acc: 0.8878 - val_loss: 0.2763 - val_acc: 0.8703
Epoch 5/30
Epoch 00005: val_loss did not improve
 - 3s - loss: 0.2262 - acc: 0.9088 - val_loss: 0.2912 - val_acc: 0.8678
Epoch 6/30
Epoch 00006: val_loss improved from 0.27635 to 0.27284, saving model to best_m.h5
 - 3s - loss: 0.2161 - acc: 0.9123 - val_loss: 0.2728 - val_acc: 0.8853
Epoch 7/30


Epoch 13/30
Epoch 00013: val_loss improved from 0.21686 to 0.21604, saving model to best_m.h5
 - 4s - loss: 0.1122 - acc: 0.9572 - val_loss: 0.2160 - val_acc: 0.9102
Epoch 14/30
Epoch 00014: val_loss did not improve
 - 3s - loss: 0.1141 - acc: 0.9601 - val_loss: 0.2430 - val_acc: 0.9152
Epoch 15/30
Epoch 00015: val_loss improved from 0.21604 to 0.20737, saving model to best_m.h5
 - 3s - loss: 0.0872 - acc: 0.9670 - val_loss: 0.2074 - val_acc: 0.9177
Epoch 16/30
Epoch 00016: val_loss did not improve
 - 3s - loss: 0.0623 - acc: 0.9771 - val_loss: 0.3331 - val_acc: 0.9102
Epoch 17/30
Epoch 00017: val_loss did not improve
 - 3s - loss: 0.0568 - acc: 0.9807 - val_loss: 0.2688 - val_acc: 0.9177
Epoch 18/30
Epoch 00018: val_loss did not improve
 - 3s - loss: 0.0320 - acc: 0.9913 - val_loss: 0.3606 - val_acc: 0.8953
Epoch 19/30
Epoch 00019: val_loss did not improve
 - 3s - loss: 0.0448 - acc: 0.9852 - val_loss: 0.3102 - val_acc: 0.9127
Epoch 20/30
Epoch 00020: val_loss did not improve
 - 3s - 

Epoch 28/30
Epoch 00028: val_loss did not improve
 - 3s - loss: 0.0053 - acc: 0.9988 - val_loss: 0.6042 - val_acc: 0.8853
Epoch 29/30
Epoch 00029: val_loss did not improve
 - 3s - loss: 0.0027 - acc: 0.9994 - val_loss: 0.7963 - val_acc: 0.8753
Epoch 30/30
Epoch 00030: val_loss did not improve
 - 3s - loss: 0.0179 - acc: 0.9936 - val_loss: 0.6660 - val_acc: 0.8653


In [9]:
import pickle
with open('../features/cnn_2_feat.pkl','wb') as fout:
    pickle.dump([train_pred,test_pred],fout)

# train feat loss
from sklearn.metrics import log_loss
print(log_loss(y,train_pred))
    
submission = pd.DataFrame()
submission['id']=test_df['id']
submission['is_iceberg']=test_pred
print(submission.head())
submission.to_csv('../results/cnn_2_sub.csv', index=False)

0.23857782251
         id  is_iceberg
0  5941774d    0.014266
1  4023181e    0.478996
2  b20200e4    0.188581
3  e7f018bb    0.997366
4  4371c8c3    0.422396
